<a href="https://colab.research.google.com/github/dnrghks5583/Way-that-set-threshold-for-setting-PMD/blob/main/ResNet56_cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pathlib
import numpy as np
from tensorflow.keras import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Input, Add, ReLU, AveragePooling2D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau

num_classes = 10

# load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
base_model = ResNet50(include_top = False, weights = 'imagenet')

x = base_model.output

shortcut = x
shortcut = Conv2D(filters = 2048, kernel_size = 1, strides = 1)(shortcut)
shortcut = BatchNormalization(axis = 3, epsilon=1.001e-5)(shortcut)

x = Conv2D(filters = 512, kernel_size = 1, strides = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 1024, kernel_size = 3, padding='SAME')(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5,)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 2048, kernel_size = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)

x = Add()([shortcut, x])
x = Activation('relu')(x)

shortcut = x
shortcut = Conv2D(filters = 2048, kernel_size = 1, strides = 1)(shortcut)
shortcut = BatchNormalization(axis = 3, epsilon=1.001e-5)(shortcut)

x = Conv2D(filters = 512, kernel_size = 1, strides = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 1024, kernel_size = 3, padding='SAME')(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5,)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 2048, kernel_size = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)

x = Add()([shortcut, x])
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)

prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=lr_schedule(0)),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])


model.fit(x_train, y_train, validation_split = 0.1, batch_size = 128, epochs = 100, callbacks=callbacks)

Learning rate:  0.001
Epoch 1/100
Learning rate:  0.001
352/352 [==============================] - 75s 102ms/step - loss: 1.3140 - accuracy: 0.5684 - val_loss: 1.5063 - val_accuracy: 0.6286
Epoch 2/100
Learning rate:  0.001
352/352 [==============================] - 34s 96ms/step - loss: 0.9746 - accuracy: 0.6891 - val_loss: 47.2593 - val_accuracy: 0.2242
Epoch 3/100
Learning rate:  0.001
352/352 [==============================] - 34s 97ms/step - loss: 1.1808 - accuracy: 0.6187 - val_loss: 52.3480 - val_accuracy: 0.2250
Epoch 4/100
Learning rate:  0.001
352/352 [==============================] - 33s 94ms/step - loss: 1.3598 - accuracy: 0.5610 - val_loss: 1.2426 - val_accuracy: 0.6012
Epoch 5/100
Learning rate:  0.001
352/352 [==============================] - 33s 95ms/step - loss: 0.9761 - accuracy: 0.6718 - val_loss: 0.9402 - val_accuracy: 0.6758
Epoch 6/100
Learning rate:  0.001
352/352 [==============================] - 34s 96ms/step - loss: 0.9214 - accuracy: 0.6943 - val_loss: 0.8

In [ ]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 7s 20ms/step - loss: 1.6422 - accuracy: 0.8079


[1.6421689987182617, 0.8079000115394592]